In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding
from tensorflow.keras.preprocessing import sequence

In [10]:
## Load the data
max_features = 10000
(X_train, y_train),(X_test, y_test) = imdb.load_data(num_words=max_features)

## Print the shape of the data
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(25000,)
(25000,)
(25000,)
(25000,)


In [11]:
## Inspect a sample review and its label
sample_review = X_train[0]
sample_label = y_train[0]
sample_review, sample_label

([1,
  14,
  22,
  16,
  43,
  530,
  973,
  1622,
  1385,
  65,
  458,
  4468,
  66,
  3941,
  4,
  173,
  36,
  256,
  5,
  25,
  100,
  43,
  838,
  112,
  50,
  670,
  2,
  9,
  35,
  480,
  284,
  5,
  150,
  4,
  172,
  112,
  167,
  2,
  336,
  385,
  39,
  4,
  172,
  4536,
  1111,
  17,
  546,
  38,
  13,
  447,
  4,
  192,
  50,
  16,
  6,
  147,
  2025,
  19,
  14,
  22,
  4,
  1920,
  4613,
  469,
  4,
  22,
  71,
  87,
  12,
  16,
  43,
  530,
  38,
  76,
  15,
  13,
  1247,
  4,
  22,
  17,
  515,
  17,
  12,
  16,
  626,
  18,
  2,
  5,
  62,
  386,
  12,
  8,
  316,
  8,
  106,
  5,
  4,
  2223,
  5244,
  16,
  480,
  66,
  3785,
  33,
  4,
  130,
  12,
  16,
  38,
  619,
  5,
  25,
  124,
  51,
  36,
  135,
  48,
  25,
  1415,
  33,
  6,
  22,
  12,
  215,
  28,
  77,
  52,
  5,
  14,
  407,
  16,
  82,
  2,
  8,
  4,
  107,
  117,
  5952,
  15,
  256,
  4,
  2,
  7,
  3766,
  5,
  723,
  36,
  71,
  43,
  530,
  476,
  26,
  400,
  317,
  46,
  7,
  4,
  2,
  1029,
  

In [12]:
## Apply padding to the data so that all reviews have the same length
maxlen = 700
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]])

In [13]:
## Train a simple RNN model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(SimpleRNN(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 700, 128)          1280000   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1313025 (5.01 MB)
Trainable params: 1313025 (5.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
## create an instabce of early stopping callback
from tensorflow.keras.callbacks import EarlyStopping
earlystopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
earlystopping

In [16]:
## Train the model with early stopping
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[earlystopping])

Epoch 1/10
625/625 [==============================] - 36s 56ms/step - loss: 882.2995 - accuracy: 0.6085 - val_loss: 0.5505 - val_accuracy: 0.6982
Epoch 2/10
625/625 [==============================] - 36s 57ms/step - loss: 0.4364 - accuracy: 0.7907 - val_loss: 0.4342 - val_accuracy: 0.7948
Epoch 3/10
625/625 [==============================] - 36s 57ms/step - loss: 0.2912 - accuracy: 0.8768 - val_loss: 0.4079 - val_accuracy: 0.8254
Epoch 4/10
625/625 [==============================] - 35s 57ms/step - loss: 0.2147 - accuracy: 0.9140 - val_loss: 0.4270 - val_accuracy: 0.8224
Epoch 5/10
625/625 [==============================] - 35s 57ms/step - loss: 0.3166 - accuracy: 0.8655 - val_loss: 0.4603 - val_accuracy: 0.8328
Epoch 6/10
625/625 [==============================] - 36s 57ms/step - loss: 0.1865 - accuracy: 0.9299 - val_loss: 0.4902 - val_accuracy: 0.8158
Epoch 7/10
625/625 [==============================] - 35s 57ms/step - loss: 0.2890 - accuracy: 0.8709 - val_loss: 0.4948 - val_accurac

In [18]:
## save model file
model.save('simple_rnn_imdb.h5')